# Рекомендации на основе содержания


1. **Загружаем датасет [MovieLens](https://grouplens.org/datasets/movielens/latest/).**

In [1]:
import pandas as pd

In [2]:
# Загрузка данных
movies_df = pd.read_csv('C:/Users/Yaros/Downloads/ml-latest-small/ml-latest-small/movies.csv')
ratings_df = pd.read_csv('C:/Users/Yaros/Downloads/ml-latest-small/ml-latest-small/ratings.csv')
tags_df = pd.read_csv('C:/Users/Yaros/Downloads/ml-latest-small/ml-latest-small/tags.csv')

In [3]:
# Предварительный просмотр данных
movies_df.head(), ratings_df.head(), tags_df.head()

(   movieId                               title  \
 0        1                    Toy Story (1995)   
 1        2                      Jumanji (1995)   
 2        3             Grumpier Old Men (1995)   
 3        4            Waiting to Exhale (1995)   
 4        5  Father of the Bride Part II (1995)   
 
                                         genres  
 0  Adventure|Animation|Children|Comedy|Fantasy  
 1                   Adventure|Children|Fantasy  
 2                               Comedy|Romance  
 3                         Comedy|Drama|Romance  
 4                                       Comedy  ,
    userId  movieId  rating  timestamp
 0       1        1     4.0  964982703
 1       1        3     4.0  964981247
 2       1        6     4.0  964982224
 3       1       47     5.0  964983815
 4       1       50     5.0  964982931,
    userId  movieId              tag   timestamp
 0       2    60756            funny  1445714994
 1       2    60756  Highly quotable  1445714996
 2       

**Фильмы (movies.csv):** Данные содержат идентификатор фильма (movieId), название (title) и жанры (genres). Жанры представлены в виде строк, где жанры разделены символом |.    
  
**Оценки (ratings.csv):** Данные включают идентификатор пользователя (userId), идентификатор фильма (movieId), оценку (rating) и временную метку (timestamp).
  
**Теги (tags.csv):** Данные содержат идентификатор пользователя (userId), идентификатор фильма (movieId), тег (tag) и временную метку (timestamp).  

2. **Предобработка данных.**

**Преобразование жанров:** Преобразуем строку жанров в список, чтобы упростить дальнейшую работу с ними.  
  
**Объединение тегов:** Для каждого фильма объединим все его теги в одну строку, чтобы упростить применение TF-IDF.  
  
**Очистка и подготовка тегов и жанров:** Удалим возможные пробелы и специальные символы, унифицируем регистр для обеспечения консистентности.

In [4]:
# Преобразование жанров в список, если они еще не преобразованы
if isinstance(movies_df['genres'].iloc[0], str):
    movies_df['genres'] = movies_df['genres'].apply(lambda x: x.split('|'))

# Обеспечение, что все теги являются строками
tags_df['tag'] = tags_df['tag'].astype(str)

# Объединение тегов для каждого фильма
tags_combined = tags_df.groupby('movieId')['tag'].apply(lambda x: ' '.join(x)).reset_index()

# Объединение тегов с данными о фильмах
movies_with_tags_df = pd.merge(movies_df, tags_combined, on='movieId', how='left')
movies_with_tags_df['tag'] = movies_with_tags_df['tag'].fillna('')  # Заполнение отсутствующих значений пустой строкой

# Предварительный просмотр результатов предобработки
movies_with_tags_df.head()

,movieId,title,genres,tag
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",pixar pixar fun
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]",fantasy magic board game Robin Williams game
2,3,Grumpier Old Men (1995),"[Comedy, Romance]",moldy old
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]",
4,5,Father of the Bride Part II (1995),[Comedy],pregnancy remake


3. **Применение TF-IDF к тегам и жанрам для извлечения фич, а также подготовка данных о средних оценках, медиане, дисперсии и других статистиках для пользователя и фильма.**

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Объединение жанров в одну строку для каждого фильма
movies_with_tags_df['genres_joined'] = movies_with_tags_df['genres'].apply(lambda x: ' '.join(x))

# Создание одной строковой фичи путем объединения тегов и жанров
movies_with_tags_df['tags_genres'] = movies_with_tags_df['tag'] + ' ' + movies_with_tags_df['genres_joined']

# Применение TF-IDF к объединенным тегам и жанрам
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(movies_with_tags_df['tags_genres'])

# Преобразование TF-IDF матрицы в DataFrame для удобства использования
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), index=movies_with_tags_df.index, columns=tfidf_vectorizer.get_feature_names_out())

tfidf_df.head()

,06,1900s,1920s,1950s,1960s,1970s,1980s,1990s,2001,250,...,york,you,younger,your,zellweger,zither,zoe,zombie,zombies,zooey
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# Уменьшим размерность
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=200)  # Выбор количества компонент
reduced_features = svd.fit_transform(tfidf_df)

4. **Подготовим данные о средних оценках, медиане, дисперсии и других статистиках для каждого пользователя и фильма.**

In [7]:
# Расчет средней оценки, медианы и дисперсии для каждого фильма
movie_stats = ratings_df.groupby('movieId')['rating'].agg(['mean', 'median', 'var']).fillna(0).reset_index()

# Расчет средней оценки, медианы и дисперсии для каждого пользователя
user_stats = ratings_df.groupby('userId')['rating'].agg(['mean', 'median', 'var']).fillna(0).reset_index()

# Подготовка названий столбцов
movie_stats.columns = ['movieId', 'movie_mean_rating', 'movie_median_rating', 'movie_rating_var']
user_stats.columns = ['userId', 'user_mean_rating', 'user_median_rating', 'user_rating_var']

# Предварительный просмотр статистик
movie_stats.head(), user_stats.head()

(   movieId  movie_mean_rating  movie_median_rating  movie_rating_var
 0        1           3.920930                  4.0          0.696990
 1        2           3.431818                  3.5          0.777419
 2        3           3.259615                  3.0          1.112651
 3        4           2.357143                  3.0          0.726190
 4        5           3.071429                  3.0          0.822917,
    userId  user_mean_rating  user_median_rating  user_rating_var
 0       1          4.366379                 5.0         0.640077
 1       2          3.948276                 4.0         0.649015
 2       3          2.435897                 0.5         4.370783
 3       4          3.555556                 4.0         1.727132
 4       5          3.636364                 4.0         0.980973)

5. **Далее объединим все подготовленные данные и подготовим их к обучению модели.**

In [8]:
# Преобразуем reduced_features в DataFrame
reduced_features_df = pd.DataFrame(reduced_features, index=movies_with_tags_df['movieId']).reset_index()

# Интеграция сниженных фич и статистик с основными данными
full_data = pd.merge(ratings_df, reduced_features_df, on='movieId', how='left')
full_data = pd.merge(full_data, movie_stats, on='movieId', how='left')
full_data = pd.merge(full_data, user_stats, on='userId', how='left')

In [9]:
# Подготовка данных для обучения модели
X = full_data.drop(['rating', 'timestamp'], axis=1)
y = full_data['rating']

6. **Обучим модель и оценим RMSE на тестовой выборке.**

In [10]:
# Преобразование имен столбцов в строки
X.columns = X.columns.astype(str)

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение модели
model = LinearRegression()
model.fit(X_train, y_train)

# Оценка модели
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"RMSE: {rmse}")

RMSE: 0.8142543739379158


- Модель успешно обучена для предсказания оценок фильмов с использованием статистик по пользователям и фильмам.  
  
- RMSE = 0.8125 демонстрирует, что модель в целом хорошо справляется с задачей предсказания оценок, но все же имеет некоторую погрешность.  
  
- Для улучшения точности модели в будущем можно рассмотреть использование дополнительных фич, таких как TF-IDF вектора тегов и жанров, или применение более сложных моделей машинного обучения.